<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Vizzu - Create Animated Pie Chart

**Tags:** #vizzu #animation #piechart #data #visualization #python

**Author:** [Alexandre Petit](https://www.linkedin.com/in/alexandre-petit-24a87a219/)

**Description:** This notebook will show how to create an animated pie chart with Vizzu. It is usefull for organizations to visualize data in an interactive way.

**References:**
- [Vizzu Documentation](https://vizzu.io/docs/getting-started/introduction)
- [Vizzu Tutorials](https://vizzu.io/tutorials)

## Input

### Import libraries

In [1]:
import naas
import pandas as pd
try:
    from ipyvizzu import Chart, Data, Config, Style, DisplayTarget
except:
    !pip install ipyvizzu --user
    from ipyvizzu import Chart, Data, Config, Style, DisplayTarget

### Setup Variables
- `file_path`: url to the csv file
- `title`: what is the graph about? "in YYYY" will be added to the title
- `category`: the column with the categories that will be displayed on the plot
- `value`: the column with the value
- `time_col`: the column with the year
- `graph_type`: the type of graph ('pie' or 'donut')
- `categories_color`: a list of hexadecimal color in the RGBA format by category. If len is not the same as the number of categories selected then it will reuse the first colors.
- `select_data`: a list  with the categories to display, "all" will include all the category but it is not recommended to display more than five categories on the same figure.
- `html_output`: The file path where the HTML output file should be created

In [2]:
#inputs
file_path = "https://raw.githubusercontent.com/alexandre-petit/oil-production/main/oil_production_pie_chart.csv"
title = "Oil Production"
category = "Entity"
value = "value"
time_col = "Year"
graph_type = "pie"
categories_color = [
    "#abababFF",
    "#ea4549FF",
    "#3562b6FF",
    "#1c9761FF",
]
select_data = [
    'Others',
    'Russia',
    'United States',
    'Saudi Arabia'
]

#outputs
html_output = "Vizzu_Create_Animated_Pie_Chart.html"

## Model

## Get data from csv

In [3]:
df = pd.read_csv(file_path)

#cleaning
df[category] = df[category].astype(str)
df[value] = df[value].astype(float)

#selecting the data for the graph
if select_data == 'all':
    pass
else:
    data_filter = (df[category].isin(select_data))
    df = df[data_filter].copy()
    
df

,Year,Entity,value
0,1900,Others,16.02614
1,1901,Others,20.74792
2,1902,Others,20.00361
3,1903,Others,29.34250
4,1904,Others,36.26236
...,...,...,...
447,2017,Saudi Arabia,6504.88430
448,2018,Saudi Arabia,6708.38900
449,2019,Saudi Arabia,6472.83940
450,2020,Saudi Arabia,6042.74900


### Create Animated Pie Chart

This function will create an animated pie chart with Vizzu.

In [6]:
#Add data to chart
data = Data()
data.add_data_frame(df)

#setting the inner radius
if graph_type == 'donut':
    y_ratio = -80
else:
    y_ratio = 0

    
#choosing the columns
config = {
    "channels": {
        "x": {"set": [category, value]},
        "y": {"range": {"min": f"{y_ratio}%"}},
        "label": {"set": [category, value]},
        "color": {"set": [category]},
    },
    "sort": "byValue",
    "coordSystem": "polar"
}

#colors, labels, padding
style = Style(
    {
        "plot": {
            "paddingLeft": 100,
            "paddingTop": 25,
            "yAxis": {
                "color": "#ffffff00",
                "label": {"paddingRight": 10},
            },
            "xAxis": {
                "title": {"color": "#ffffff00"},
                "label": {
                    "color": "#ffffff00",
                    "numberFormat": "grouped",
                },
            },
            "marker": {
                #colorPalette take a string of hexadecimal color separated by a whitespace
                #for simplicity, we will join the list of hexadecimal color defined before
                "colorPalette": " ".join(categories_color)

            },
        },
    }
)

#Initialize Chart
chart = Chart(width="640px", height="360px", display=DisplayTarget.MANUAL)
chart.animate(data, style)

#Get time range
min_date = df[time_col].min()
max_date = df[time_col].max()

# This part is looping over the time period
for year in range(min_date, max_date + 1):
    config["title"] = title + f" in {year}"
    chart.animate(
        Data.filter(f"parseInt(record.{time_col}) == {year}"),
        Config(config),
        duration=0.5,
        x={"easing": "linear", "delay": 0},
        y={"delay": 0},
        show={"delay": 0},
        hide={"delay": 0},
        title={"duration": 0, "delay": 0},
    )

#Display Chart
chart.show()

## Output

### Export chart in HTML

In [5]:
chart._showed = False
rawhtml = chart._repr_html_()

with open(html_output, "w", encoding="utf8") as file_desc:
    file_desc.write(rawhtml)